In [57]:
import pandas as pd
import numpy as np
import csv
import ast
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import time
import logging
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(
    filename='scraping_missing.log',
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [2]:
scraped_details_df = pd.read_csv('aryaman/csv/scraping/scraping_details.csv',encoding='utf-8')
products = pd.read_csv('aryaman/csv/categories_and_products/products.csv')
missing_details_df = pd.read_csv("aryaman/csv/scraping/MissingDetailsChart.csv")

In [3]:
scraped_details_df['Images'] = scraped_details_df['Images'].apply(lambda item: ast.literal_eval(item) if pd.notna(item) else item)
scraped_details_df['productOverview'] = scraped_details_df['productOverview'].apply(lambda item: ast.literal_eval(item) if pd.notna(item) else item)
scraped_details_df['featureBullets'] = scraped_details_df['featureBullets'].apply(lambda item: ast.literal_eval(item) if pd.notna(item) else item)

In [6]:
bullets_and_overview_same_df = missing_details_df[missing_details_df['featureBullets']== missing_details_df['Overview']]
diff_bullets_and_overview_df = pd.concat([bullets_and_overview_same_df,missing_details_df]).drop_duplicates(keep=False)
diff_bullets_and_overview_df[['featureBullets','Overview']].value_counts()

featureBullets  Overview
False           True        259
True            False        67
Name: count, dtype: int64

In [8]:
merged_df = pd.merge(products,missing_details_df[(missing_details_df['featureBullets']==True) & (missing_details_df['Overview']==True)],on='ASIN',how='inner')

In [75]:
webdriver_path = "..\chromedriver.exe"
cService = webdriver.ChromeService(executable_path=webdriver_path)
# chrome_options = Options()
# chrome_options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=cService)

In [16]:
# with open('scraping_missing_details_featureBullets_Overview.csv','w',newline='',encoding='utf-8') as file:
#     writer = csv.DictWriter(file,fieldnames=['ASIN',  'productOverview', 'featureBullets'])
#     writer.writeheader()

In [17]:
# pd.DataFrame({"blacklisted_asins":[]}).to_csv('blacklisted2.csv',index=False)

In [18]:
def filter_unscraped_iter(products_links_df,scraped_details_df):
    return list(products_links_df[~(products_links_df['ASIN'].isin(scraped_details_df['ASIN']))].itertuples(index=False))

In [77]:
scraped_missing_details_df = pd.read_csv('scraping_missing_details_featureBullets_Overview.csv',encoding='utf-8')
products_links = merged_df[['ASIN','Product Link']]
products_links.reset_index(inplace=True,drop=True)
blacklisted_asins = (pd.read_csv('blacklisted2.csv'))['blacklisted_asins'].tolist()
missing_details_update_df = pd.read_csv("MissingDetailsUpdated.csv")
no_of_items_to_scrape = 1000
how_many_scraped = 0
with open('scraping_missing_details_featureBullets_Overview.csv','a',newline='',encoding='utf-8') as file:
    writer = csv.DictWriter(file,fieldnames=['ASIN','productOverview','featureBullets'])
    iterator_to_loop = filter_unscraped_iter(products_links,scraped_missing_details_df)
    for link in iterator_to_loop:
        if link.ASIN in blacklisted_asins:
            continue
        driver.get(link._1)
        time.sleep(1)
        try:
            productFactsDiv = driver.find_element(By.ID,'productFactsDesktop_feature_div')
            try:
                overview = dict()
                headings = productFactsDiv.find_elements(By.TAG_NAME,'h3')
                product_details_heading = None
                for heading in headings:
                    if ((driver.execute_script("return arguments[0].textContent",heading)).strip() == 'Product details'):
                        product_details_heading = heading
                if product_details_heading != None:
                    missing_details_update_df.loc[missing_details_update_df['ASIN']==link.ASIN,'Overview'] =  False
                    logging.info(f"missing details updated {link.ASIN}")
                    nextSibling = driver.execute_script("return arguments[0].nextElementSibling",product_details_heading)
                    while( nextSibling.tag_name == 'div' and 'product-facts-detail' in nextSibling.get_attribute("class")):
                        spans = nextSibling.find_elements(By.CSS_SELECTOR,"span.a-color-base")
                        if len(spans) == 2:
                            if len(spans[0].find_elements(By.CSS_SELECTOR,"*")) == len(spans[1].find_elements(By.CSS_SELECTOR,"*")) == 0:
                                overview[driver.execute_script("return arguments[0].textContent",spans[0])] = driver.execute_script("return arguments[0].textContent",spans[1])
                        nextSibling = driver.execute_script("return arguments[0].nextElementSibling",nextSibling)
            except NoSuchElementException:
                print("over view not found",link.ASIN)
                logging.info(f"overview not found {link.ASIN}")
            except Exception as e:
                print(f"Exception in headings {link.ASIN}",e)
                break
            try:
                bullets_text = list()
                list_of_bullets = productFactsDiv.find_element(By.TAG_NAME,'ul')
                bullets = list_of_bullets.find_elements(By.TAG_NAME,'li')
                if len(bullets) == 0:
                    raise ValueError
                missing_details_update_df.loc[missing_details_update_df['ASIN']==link.ASIN,'featureBullets'] =  False
                logging.info(f"missing details updated {link.ASIN}")
                for bullet in bullets:
                    spans = bullet.find_elements(By.TAG_NAME,'span')
                    if len(spans) == 1:
                        if len(spans[0].find_elements(By.CSS_SELECTOR,"*")) == 0:
                            bullets_text.append((driver.execute_script("return arguments[0].textContent",spans[0])))
                
            except NoSuchElementException:
                print("bullets not found ",link.ASIN)
                logging.info(f"bullets not found {link.ASIN}")
            except Exception as e:
                print(f"Exception in bullets {link.ASIN}",e)
                break
            if len(overview)>0 or len(bullets_text) > 0:
                if len(overview) == 0:
                    overview = None
                if len(bullets_text) == 0:
                    bullets_text = None
                new_entry = {'ASIN':link.ASIN,'productOverview':overview,'featureBullets':bullets_text}
                writer.writerow(new_entry)
                file.flush()
            missing_details_update_df.to_csv('MissingDetailsUpdated.csv',index=False)
            no_of_items_to_scrape -= 1
            if no_of_items_to_scrape == 0:
                print('break normally')
                break
            how_many_scraped += 1
            if how_many_scraped == 50:
                print(1000-no_of_items_to_scrape)
                how_many_scraped = 0
            
        except ValueError as e:
            print(f"value error occurred,breaking {link.ASIN}",e)
            break
        except NoSuchElementException:
            # print("Blacklisting")
            blacklisted_asins.append(link.ASIN)
            pd.DataFrame({'blacklisted_asins':blacklisted_asins}).to_csv('blacklisted2.csv',index=False)
            logging.warning(f"blacklisted - {link.ASIN}")
        except Exception as e:
            print(f"some exception occurred,breaking {link.ASIN}",e)
            break
missing_details_update_df.to_csv('MissingDetailsUpdated.csv',index=False)
logging.info(f"missing details flushed to file")